## [Is the deconvolution layer the same as a convolutional layer](https://arxiv.org/pdf/1609.07009.pdf)



In [ ]:
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import numpy as np

import sys
sys.path.append('C:/Anaconda3/envs/tensorflow/Lib/site-packages')
import tensorflow as tf

%matplotlib inline

* [pretty table show](https://matplotlib.org/gallery/images_contours_and_fields/image_annotated_heatmap.html#sphx-glr-gallery-images-contours-and-fields-image-annotated-heatmap-py)

In [ ]:
def show(mts):
    if(type(mts) != list):
        mts = [mts]

    for i, mt in enumerate(mts):
        X = mt.shape[0]
        Y = mt.shape[1]
        fig, ax = plt.subplots(figsize=(Y, X))        
        im = ax.imshow(mt)

        # We want to show all ticks...
        ax.set_xticks(np.arange(Y))
        ax.set_yticks(np.arange(X))
        # ... and label them with the respective list entries
        ax.set_xticklabels([y for y in range(Y)])
        ax.set_yticklabels([x for x in range(X)])

        # Rotate the tick labels and set their alignment.
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
                 rotation_mode="anchor")

        # Loop over data dimensions and create text annotations.
        for x in range(X):
            for y in range(Y):
                text = ax.text(y, x, mt[x, y], fontweight='bold',fontsize='x-large',
                               ha="center", va="center", color="w")

        ax.set_title('matrix%d'%(i))
        fig.tight_layout()
    plt.show()

def expand(mt, stride):
    X = mt.shape[0]
    Y = mt.shape[1]
    ret = np.zeros((X*stride,Y*stride))
    for x in range(X):
        for y in range(Y):
            ret[x*stride, y*stride] = mt[x,y]
    return ret

def conv2d(data, kernel):
    data = data.astype(np.float64)
    kernel = kernel.astype(np.float64)
    data = data.reshape(1, data.shape[0], data.shape[1], 1)
    kernel = kernel.reshape(kernel.shape[0], kernel.shape[1], 1, 1)
    input = tf.Variable(data)
    filter = tf.Variable(kernel)

    result = None

    op = tf.nn.conv2d(input, filter, strides=[1, 1, 1, 1], padding='SAME')
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)

        result = sess.run(op)
    return result

def deconv2d(data, kernel, stride):
    data = data.astype(np.float64)
    kernel = kernel.astype(np.float64)
    data = data.reshape(1, data.shape[0], data.shape[1], 1)
    kernel = kernel.reshape(kernel.shape[0], kernel.shape[1], 1, 1)
    input = tf.Variable(data)
    filter = tf.Variable(kernel)
    
    output_shape = [1, (data.shape[0]+1)*stride, (data.shape[1]+1)*stride, 1]

    result = None

    op = tf.nn.conv2d_transpose(input, filter, 
                                output_shape=output_shape,
                                strides=[1, stride, stride, 1], padding='VALID')
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)

        result = op.eval(session=sess)
    return result

def split(kernel, stride):
    X = kernel.shape[0]
    Y = kernel.shape[1]
    X0 = int(X/stride)
    Y0 = int(Y/stride)
    kernels = []
    for s in range(stride*stride):
        k = np.zeros((X0,Y0))
        for x in range(X0):
            for y in  range(Y0):
                x0 = int(s/stride) + x*stride 
                y0 = int(s%stride) + y*stride 
                k[x,y] = kernel[x0, y0]
        kernels.append(k)
    return kernels

def periodic_shuffle(datas, stride):
    s = [0, 0]
    datas.reverse()
    X = datas[0].shape[0]
    Y = datas[0].shape[1]
    ret = np.zeros((X*stride,Y*stride))
    for x in range(X):
        for y in range(Y):
            for x0 in range(stride):
                for y0 in range(stride):
                    ret[x*stride+x0,y*stride+y0] = datas[x0*stride+y0][x,y]
    return ret 

In [ ]:
D=4
data = np.random.randint(1,10,size=(D,D))
#data1 = np.pad(data,[0,1],mode='constant')
show(data)

In [ ]:
S=2
data2 = expand(data, S)
#data2 = np.pad(data2,[0,1],mode='constant')
show(data2)

In [ ]:
K=4
kernel = np.random.randint(1,10,size=(K,K))
show(kernel)

In [ ]:
kernels = split(kernel,S)
show(kernels)

In [ ]:
result = conv2d(data2, kernel)
result = result.reshape(D*S,D*S)
show(result)


In [ ]:
#result2 = deconv2d(data, kernel, S)
#result2 = result2.reshape(D*S,D*S)
#show(result2)

In [ ]:
results = []
for k in kernels:
    result = conv2d(data, k)
    result = result.reshape(D,D)
    results.append(result)
show(results)

In [ ]:
result = periodic_shuffle(results, S)
show(result)